In [ ]:
def my_affinity(d1,d2,i,j,dist):
    distance = dist(d1[i],d2[j])
     
    if i == 0 and j == 0:
        return distance
    elif i == 0:
        couples = [[i,j-1]]
    elif j == 0:
        couples = [[i-1,j]]
    else:
        couples = [[i,j-1],[i-1,j],[i-1,j-1]]
        
    distance += np.amin([ my_affinity(d1,d2,e[0],e[1],dist) for e in couples])
    return distance

#### Compute the dtw distance (O(n^2))

#### Comparison of the results of fastdtw ant dtw

In [ ]:
print(my_affinity_wrapper(discretized_pixel_trajectories[50][0] ,discretized_pixel_trajectories[100][0]))

In [ ]:
distance, path = fastdtw(discretized_pixel_trajectories[50][0], discretized_pixel_trajectories[100][0], dist=euclidean)
print(distance)

### Compute the distance matrix

In [ ]:
def distance_matrix(trajectories):
    l =len(trajectories)
    D = np.zeros((l,l))
    for i in range(l):
        for j in range(i,l):
            #D[i,j] = my_affinity_wrapper(trajectories[i],trajectories[j])
            D[i,j],p = fastdtw(trajectories[i],trajectories[j], dist=euclidean)
    return D    

In [ ]:
times=[]
start = time.time()
D = distance_matrix(discretized_pixel_trajectories[:][0])
end = time.time()-start
times.append(end)
print(end)


In [ ]:
model = DBSCAN(eps=0.5, min_samples=3,metric='precomputed').fit(D)
model = AffinityPropagation(damping=0.9, max_iter=200, convergence_iter=15, copy=True, preference=None, affinity='precomputed').fit(D)
model = SpectralClustering(n_clusters=15,affinity='precomputed' ).fit(D)
clusters = model.fit_predict(D)
nb_clusters = np.max(clusters)
print(nb_clusters)
print((clusters.tolist()).count(-1))
print(clusters[0:200])

In [ ]:
color = colors(nb_clusters+1)
cap = cv2.VideoCapture('laurier.avi')
while(cap.isOpened()):
    ret, frame = cap.read()

    for i,t in enumerate(pixel_trajectories):
        pts = np.array(t, np.int32)
        pts = pts.reshape((-1,1,2))
    #print(pts)
        #print(clusters[i])
    #frame = cv2.circle(frame,(151,46), 50, (0,0,255), -1)
        if clusters[i] == 5:
            c = color[clusters[i]]
            frame = cv2.polylines(frame,[pts],False,c)
    cv2.imshow('frame',frame)
    #cv2.waitKey(10000)
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()